In [1]:
from langchain_openai.llms import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())  

True

In [2]:
llm = OpenAI()

In [3]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário:
    {pergunta}
"""
)

In [4]:
print( prompt_template.format(pergunta="O que é um SaaS?"))


    Responda a seguinte pergunta do usuário:
    O que é um SaaS?



In [5]:
prompt_template = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {pergunta}
"""
)

In [6]:
print( prompt_template.format(pergunta="O que é um SaaS?", n_palavras=15))


    Responda a seguinte pergunta do usuário em até 15 palavras:
    O que é um SaaS?



In [7]:
prompt_template = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {pergunta}
""",
partial_variables={"n_palavras": 10}
)

In [8]:
print( prompt_template.format(pergunta="O que é LangChain?"))


    Responda a seguinte pergunta do usuário em até 10 palavras:
    O que é LangChain?



### Utilizando múltiplos prompts

In [9]:
from langchain.prompts import PromptTemplate

template_word_cont = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {pergunta}
"""
)

template_line_count = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em até {n_linhas} linhas:
    {pergunta}
"""
)

template_idioma = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário em {idioma}.
"""
)

template_final = (template_word_cont + template_line_count + template_idioma +
"    Responda a pergunta seguindo as instruções {pergunta}."
)

In [10]:
print(template_final)

input_variables=['idioma', 'n_linhas', 'n_palavras', 'pergunta'] template='\n    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:\n    {pergunta}\n\n    Responda a seguinte pergunta do usuário em até {n_linhas} linhas:\n    {pergunta}\n\n    Responda a seguinte pergunta do usuário em {idioma}.\n    Responda a pergunta seguindo as instruções {pergunta}.'


In [11]:
prompt_final = template_final.format(n_palavras=15, n_linhas=2, idioma="inglês", pergunta="O que é LangChain?")
llm.invoke(prompt_final)

'\n\n    LangChain is a blockchain platform designed for creating and managing multilingual content.'

In [12]:
print(prompt_final)


    Responda a seguinte pergunta do usuário em até 15 palavras:
    O que é LangChain?

    Responda a seguinte pergunta do usuário em até 2 linhas:
    O que é LangChain?

    Responda a seguinte pergunta do usuário em inglês.
    Responda a pergunta seguindo as instruções O que é LangChain?.


### Template para chat

In [13]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template("Essa é minha dúvida: {duvida}")
chat_template.format_messages(duvida="Quem é você?")

[HumanMessage(content='Essa é minha dúvida: Quem é você?')]

In [14]:
chat_template = ChatPromptTemplate.from_messages([
    "system", "Você é um assistente irônico e se chama {nome_assistente}.",
    ("human", "Olá, como vai?"),
    ("ai", "Estou bem, como posso lhe ajudar?"),
    ("human", "{pergunta}")
])

In [15]:
chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='system')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], template='Você é um assistente irônico e se chama {nome_assistente}.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Olá, como vai?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Estou bem, como posso lhe ajudar?')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], template='{pergunta}'))])

In [16]:
chat_template.format_messages(nome_assistente="Liorl", pergunta="Qual é o seu nome?")

[HumanMessage(content='system'),
 HumanMessage(content='Você é um assistente irônico e se chama Liorl.'),
 HumanMessage(content='Olá, como vai?'),
 AIMessage(content='Estou bem, como posso lhe ajudar?'),
 HumanMessage(content='Qual é o seu nome?')]

In [17]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

chat.invoke(chat_template.format_messages(nome_assistente="Liorl", pergunta="Qual é o seu nome?"))

AIMessage(content='Meu nome é Liorl, o assistente irônico. Como posso ser sarcástico hoje?', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 64, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5d8e98ad-778f-4584-a69c-5ef8f3df4102-0')

### Few shot prompting


In [18]:
exemplos = [
    {"pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 
""", 
    }, 
    {"pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 
""", 
    }, 
    {"pergunta": "Quem foi o pai de Napoleão Bonaparte?",
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte? 
Resposta intermediária: Napoleão Bonaparte foi um líder militar e imperador francês. 
Pergunta de acompanhamento: Quem foi o pai de Napoleão Bonaparte? 
Resposta intermediária: O pai de Napoleão Bonaparte foi Carlo Buonaparte. 
Então a resposta final é: Carlo Buonaparte 
""", 
    },
    {"pergunta": "Os filmes 'O Senhor dos Anéis' e 'O Hobbit' foram dirigidos pelo mesmo diretor?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem dirigiu 'O Senhor dos Anéis'? 
Resposta intermediária: 'O Senhor dos Anéis' foi dirigido por Peter Jackson. 
Pergunta de acompanhamento: Quem dirigiu 'O Hobbit'? 
Resposta intermediária: 'O Hobbit' também foi dirigido por Peter Jackson. 
Então a resposta final é: Sim 
""",
    },
]

In [19]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["pergunta", "resposta"],
    template="Pergunta {pergunta}\n{resposta}"
)

example_prompt.format(**exemplos[0])

'Pergunta Qual é a maior montanha do mundo, o Monte Everest ou o K2?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Qual é a altura do Monte Everest? \nResposta intermediária: O Monte Everest tem 8.848 metros de altura. \nPergunta de acompanhamento: Qual é a altura do K2? \nResposta intermediária: O K2 tem 8.611 metros de altura. \nEntão a resposta final é: Monte Everest \n'

In [20]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix="Pergunta: {input}",
    input_variables=["input"]
    
)

In [21]:
print(prompt.format(input="Quem é melhor, o Messi ou o Cristiano Ronaldo?"))

Pergunta Qual é a maior montanha do mundo, o Monte Everest ou o K2?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 


Pergunta Quem nasceu primeiro, Charles Darwin ou Albert Einstein?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 


Pergunta Quem foi o pai de Napoleão Bonaparte?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte?

In [22]:
llm.invoke(prompt.format(input="Quem fez mais goals, Messi ou Cristiano Ronaldo?"))

'\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos goals Messi fez? \nResposta intermediária: Segundo dados do site Globo Esporte, Messi fez 736 goals em sua carreira. \nPergunta de acompanhamento: Quantos goals Cristiano Ronaldo fez? \nResposta intermediária: De acordo com o mesmo site, Cristiano Ronaldo fez 733 goals em sua carreira. \nEntão a resposta final é: Messi'